In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = torch.device("cpu")
if torch.cuda.is_available():
    print("CUDA is available!")
    device = torch.device("cuda")
else:
    print("CUDA not available!")

# if torch.backends.mps.is_available():
#     print("MPS is available!")
#     device = torch.device("mps")
# else:
#     print("MPS not available!")

CUDA is available!


In [3]:
# load text file
with open('/content/input.txt', 'r') as f:
    text = f.read()

print(f"Total text size: {len(text)}\n")
print('Text:\n'+text[:300])

Total text size: 1115394

Text:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [4]:
# defining context length, batch size, and embedding dimensions
context_len = 256
batch_size = 64
embedding_dim = 384

In [5]:
# define character level vocabulary
char_l = list(set(text))
char_l.sort()

vocab_size = len(char_l)
print(f"Vocab: {char_l}")
print(f"Vocab size: {vocab_size}")

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Vocab size: 65


In [6]:
# ch to index and index to ch maps
stoi = {ch:i for i,ch in enumerate(char_l)}
itos = {i:ch for i,ch in enumerate(char_l)}
print('Index corresponding to "!" : ' + str(stoi['!']))
print('Character corresponding to index 2 : ' + itos[2])

Index corresponding to "!" : 2
Character corresponding to index 2 : !


In [7]:
# encoder and decoder functions
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello"))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [8]:
# training and validations sets
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data)*0.9)

train_data = data[:n]
val_data = data[n:]

print(f"Train size: {len(train_data)}, Val size: {len(val_data)}")

Train size: 1003854, Val size: 111540


In [9]:
# data loader
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data)-context_len, (batch_size,))
    x = torch.stack([data[i:i+context_len] for i in ix])
    y = torch.stack([data[i+1:i+1+context_len] for i in ix])
    x = x.to(device)
    y = y.to(device)

    return x, y

In [10]:
# layer hyperparameters
dropout_r = 0.2
num_heads = 6
num_layers = 6

In [11]:
# self attention head
class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embedding_dim, head_size, bias=False)
        self.query = nn.Linear(embedding_dim, head_size, bias=False)
        self.value = nn.Linear(embedding_dim, head_size, bias=False)

        self.register_buffer('tril', torch.tril(torch.ones(context_len, context_len)))
        self.dropout = nn.Dropout(dropout_r)

    def forward(self, x):
        B,T,C = x.shape # Batch, time_step, channels (embedding)
        k = self.key(x) # B,T,hs
        q = self.query(x) # B,T,hs

        w = q @ k.transpose(-2,-1) * k.shape[-1]**(-0.5) # (B,T,hs) @ (B,hs,T) -> (B,T,T)
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # B,T,T (replace upper triangular values with -inf before softmax)
        w = torch.softmax(w, dim=-1) # B,T,T
        w = self.dropout(w)

        v = self.value(x) # B,T,hs
        out = w @ v # (B,T,T) @ (B,T,hs) -> (B,T,hs)
        return out

In [12]:
# multihead attention
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embedding_dim)
        self.dropout = nn.Dropout(dropout_r)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out

In [13]:
# one block of transformer network
class Block(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super().__init__()
        head_size = embedding_dim // num_heads
        self.ffwd = nn.Sequential(
            nn.Linear(embedding_dim, 4*embedding_dim),
            nn.ReLU(),
            nn.Linear(4*embedding_dim, embedding_dim),
            nn.Dropout(dropout_r),
        )
        self.attention = MultiHeadAttention(num_heads, head_size)
        self.norm1 = nn.LayerNorm(embedding_dim)
        self.norm2 = nn.LayerNorm(embedding_dim)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.ffwd(self.norm2(x))
        return x

In [54]:
# character level transformer network model
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding_table = nn.Embedding(context_len, embedding_dim)
        self.blocks = nn.Sequential(*[Block(embedding_dim, num_heads=num_heads) for _ in range(num_layers)])
        self.ln_final = nn.LayerNorm(embedding_dim)
        self.lm_head = nn.Linear(embedding_dim, vocab_size)
        self.apply(self._init_weights)

    # Transformers use std=0.2 in Gaussian weight initialization for better performance
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape # same shape as targets

        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)

        x = self.blocks(x) # (B,T,C)
        x = self.ln_final(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is total generated indices till now
        for i in range(max_new_tokens):
            idxb = idx[:, -context_len:]  # crop last context_length of tokens
            logits, loss = self(idxb)  # get predictions and loss
            logits = logits[:,-1,:] # reshape to (B,C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1).to(device)

            idx = torch.cat((idx, idx_next), dim=1)

        return idx


In [55]:
model = LanguageModel()
m = model.to(device)
# number of parameters
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

10.788929 M parameters


In [60]:
eval_iters = 200
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [57]:
# training loop
learning_rate = 3e-4
eval_interval = 500
max_iters = 5000

optim = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [58]:
best_val_loss = float('inf')
best_model_state = None

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

        # Check best val loss
        val_loss = losses['val']
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = m.state_dict()

    # Sample a batch from the training set
    x, y = get_batch('train')

    # Evaluate the loss
    logits, loss = m(x, y)

    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

# Save best and last parameters
if best_model_state is not None:
    torch.save(best_model_state, 'best.pth')
torch.save(m.state_dict(), 'last.pth')

step 0: train loss: 4.1941, val loss: 4.1906
step 500: train loss: 1.7707, val loss: 1.9106
step 1000: train loss: 1.4091, val loss: 1.6298
step 1500: train loss: 1.2796, val loss: 1.5363
step 2000: train loss: 1.1995, val loss: 1.5062
step 2500: train loss: 1.1414, val loss: 1.4939
step 3000: train loss: 1.0878, val loss: 1.4912
step 3500: train loss: 1.0316, val loss: 1.4990
step 4000: train loss: 0.9786, val loss: 1.5119
step 4500: train loss: 0.9298, val loss: 1.5154
step 4999: train loss: 0.8813, val loss: 1.5439


In [62]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))


RATCLIFF:
Now my lord.

QUEEN ELIZABETH:
Straw with Richard not your mistress' sorrow.

KING RICHARD III:
Ratcliff!

RICHARD:

RATCLIFF:
Art thou far'st, my sovereign, lord, my son:
I am a winr that is sort frivord.
That done me doth a spird, answer to me.

KING RICHARD III:
So then the cause of my tale in pastimage.

VIRCHARD:
Had I aid, and my lord that speaks too have
You as a great dinning hate it on tomb.

QUEEN ELIZABETH:
Say not so horself the injuret: strifes
Which but God but stand me h
